In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# pip install fastparquet

In [ ]:
# Filtering out the columns that are not needed
cols_to_keep = ['activity_id', 'timestamp', 'total_power', 'air_power', 'form_power', 'heart_rate', 'cadence', 'stride_length', 'speed', 'vertical_oscillation','ground_time','peak_vertical_grf','leg_spring','grade','temperature','humidity','distance','duration','apparel_model_brand','apparel_model_name','apparel_size_primary','apparel_size_region','env_temperature','env_humidity','weight_kg','elevation','flight_time_ms','stride_time_ms','duty_factor_perc','vertical_stiffness_kN_m','altitude_change_m','stryd_air_power_w','df_activities_user_id','ts','critical_power','stryds','env_dew_point','env_wind_bearing','env_wind_speed','env_wind_gust','number_steps','bsa_m2','fsa_m2','mmp_10','mmp_30','mmp_60','mmp_120','mmp_300','mmp_600','mmp_1800','mmp_3600','mmp_5400','mmp_7200','mmp_10_w_kg','mmp_30_w_kg','mmp_60_w_kg','mmp_120_w_kg','mmp_300_w_kg','mmp_600_w_kg','mmp_1800_w_kg','mmp_3600_w_kg','mmp_5400_w_kg','mmp_7200_w_kg','age','gender']

In [ ]:
# full_final_data = pd.read_csv('full_final_data.csv')
full_final_data = pd.read_parquet('full_final_data.parquet', engine='fastparquet')

filtered_df = full_final_data.copy()
filtered_df = filtered_df[cols_to_keep].dropna()
filtered_df = filtered_df[filtered_df['gender']!='other']
filtered_df['apparel_model'] = filtered_df["apparel_model_brand"] + ' ' + filtered_df["apparel_model_name"]
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
# # Additional Filters, if necessary

# #filtering the data
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['speed'].between(1, 10)).all())
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['vertical_oscillation'].between(1, 25)).all())
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['ground_time'].between(100, 420)).all())

# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['grade'].between(-0.20, 0.20)).all())
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['leg_spring'].between(1, 20)).all())
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['cadence'].between(120, 210)).all())
# filtered_df = filtered_df.groupby('activity_id').filter(lambda x: (x['stride_length'].between(0.3, 2.5)).all())

In [ ]:
# Adding the shoe type to the data
    # Step 1: Define your lists of super shoes and non-super shoes
super_shoes = [
    'Adidas Adizero Adios Pro', 'Adidas Adizero Pro', 'Adidas Adizero Prime X',
    'Adidas Adizero Prime', 'Adidas Adizero Adios Pro 2', 'Altra Vanish Tempo',
    'Altra Vanish Carbon', 'ASICS Metaspeed Sky', 'ASICS Magic Speed', 'ASICS Metaspeed Edge',
    'ASICS MetaRacer', 'Brooks Hyperion Elite', 'HOKA ONE ONE Carbon X', 'HOKA ONE ONE Rocket X',
    'New Balance FuelCell SuperComp Trainer', 'New Balance FuelCell SuperComp Elite',
    'New Balance FuelCell RC Elite', 'Nike Vaporfly Next %', 'Nike Alphafly Next %',
    'Nike Air Zoom Tempo Next %', 'Nike Vaporfly 4%', 'On Cloudboom', 'On Cloudboom Echo',
    'Saucony Endorphin Pro'
]

non_super_shoes = [
    'Adidas Ultraboost', 'Adidas Solar Glide', 'Adidas Solar Boost', 'Adidas SL20',
    'Adidas Supernova', 'Adidas Ultraboost 21', 'Altra Lone Peak', 'Altra Escalante', 
    'Altra Torin', 'ASICS Novablast', 'ASICS GT 2000', 'ASICS Gel Nimbus', 'ASICS Gel Cumulus', 
    'ASICS DynaFlyte', 'ASICS Gel Kayano', 'ASICS Gel DS Trainer', 'ASICS Gel Noosa Tri',
    'ASICS GlideRide', 'ASICS Superblast', 'ASICS EvoRide', 'ASICS Noosa Tri', 'ASICS Noosa',
    'Brooks Ghost', 'Brooks Launch', 'Brooks Adrenaline', 'Brooks Hyperion Tempo', 'Brooks Levitate', 
    'Brooks Glycerin', 'Brooks Ricochet', 'Brooks Defyance', 'HOKA ONE ONE Clifton', 
    'HOKA ONE ONE Mach', 'HOKA ONE ONE Speedgoat', 'HOKA ONE ONE Rincon', 'HOKA ONE ONE Arahi',
    'HOKA ONE ONE Challenger', 'HOKA ONE ONE Bondi', 'HOKA ONE ONE Tecton X',
    'New Balance Fresh Foam 1080', 'New Balance 880', 'New Balance FuelCell Rebel', 
    'New Balance 1500', 'New Balance 890', 'Nike Pegasus', 'Nike Pegasus Turbo', 'Nike Infinity Run', 
    'Nike Invincible', 'Nike Vomero', 'Nike Terra Kiger', 'Nike Pegasus Shield', 'On Cloudflow', 
    'On Cloudflyer', 'On Cloudmonster', 'On Cloudstratus', 'On Cloudsurfer', 'Saucony Endorphin Speed', 
    'Saucony Kinvara', 'Saucony Ride', 'Saucony Triumph', 'Saucony Peregrine', 'Saucony Endorphin Shift', 
    'Saucony Guide', 'Saucony Fastwitch', 'Saucony Munich'
]


filtered_df = filtered_df[filtered_df['apparel_model'].isin(super_shoes+non_super_shoes)].reset_index(drop=True)

    # Step 2: Define a function to categorize each shoe
def categorize_shoe(model):
    if model in super_shoes:
        return 'Super Shoe'
    elif model in non_super_shoes:
        return 'Non-Super Shoe'
    else:
        return 'Unknown'  # In case the model is not in either list

    # Step 3: Apply the function to your DataFrame to create a new column
filtered_df['shoe_type'] = filtered_df['apparel_model'].apply(categorize_shoe)

filtered_df = filtered_df[filtered_df["apparel_size_primary"]!= 255]
def convert_size(size, region, target_region="us"):
    if region == "uk":
        if target_region == "us":
            return size + 2
        elif target_region == "eu":
            return size + 33
    elif region == "us":
        if target_region == "uk":
            return size - 2
        elif target_region == "eu":
            return size + 31
    elif region == "eu":
        if target_region == "us":
            return size - 31
        elif target_region == "uk":
            return size - 33
    return size  # Return the size unchanged if no conversion is needed

# Apply conversion
filtered_df['normalised_shoe_size'] = filtered_df.apply(lambda row: convert_size(row['apparel_size_primary'], row['apparel_size_region'], target_region="us"), axis=1)

# Adding the intensity to the data
filtered_df["relative_total_power"] = filtered_df["total_power"] / filtered_df["weight_kg"]
filtered_df["max_mmp600"] = filtered_df.groupby('df_activities_user_id')['mmp_600_w_kg'].transform('max')
filtered_df["intensity"] = filtered_df["relative_total_power"]/filtered_df["max_mmp600"]

In [ ]:
# Adding quarters to the data
def assign_quartiles(group):
    group = group.sort_values(by='timestamp')  # Sort each group by timestamp
    try:
        group['quartile'] = pd.qcut(group['timestamp'], q=4, labels=[1, 2, 3, 4], duplicates='drop')
    except ValueError as e:
        # Handle cases where fewer than 4 bins are created due to duplicates
        group['quartile'] = pd.qcut(group['timestamp'], q=len(group['timestamp'].unique()), labels=False) + 1
    group['heart_rate_diff'] = group['heart_rate'].diff()
    return group

filtered_df = filtered_df.groupby('activity_id').apply(assign_quartiles)

# Define a custom aggregation function
def custom_agg(x):
    if pd.api.types.is_numeric_dtype(x):
        return np.mean(x)
    else:
        return x.mode()[0] if not x.mode().empty else np.nan
    
all_data_full = filtered_df.reset_index(drop=True).groupby(["activity_id",'quartile']).agg(custom_agg).reset_index()

In [ ]:
# Save the data
# all_data_full.to_csv('all_data_full.csv', index=False)

# Save the data according to brand
# adidas_data_full = all_data_full[all_data_full['apparel_model_brand']=='Adidas']
# adidas_data_full.to_csv('adidas_data_full.csv', index=False)